# Neuromap Project Neurohackademy 2019

In [3]:
# install libraries/ set up script
import pandas as pd
import numpy as np
import folium
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="neuromap")

In [4]:
# sets the defaults for viewing the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
#identify missing values
missing_values = ["  ", "-"]
df = pd.read_csv('NeuroMap-38responses.csv', na_values = missing_values, encoding='latin-1')

#view data
df.head(20)

,Timestamp,Do you consent to have your information used to create a visualisation of geographic and research area career trajectory?,What is your name (optional)?,"Where were you born (city, state/region, country)?",What is your year of birth (yyyy)?,What is your gender?,Name of undergraduate degree?,"Where did you do your undergraduate degree? (city, state/region, country).",At what institution did you do your undergraduate degree?,"Undergraduate research topic area (3-5 key words e.g. population, methodology and brain region/cognitive function)?",Do you have any post-undergraduate experience (research assistant/lab manager)?,"Where did you work as a lab manager / research assistant? (city, state/region, country).",At what institution did you work as a lab manager / research assistant?,"Lab manager / research assistant research topic area (3-5 words e.g. population, methodology and brain region/cognitive function)?",Do you have a master's degree or are you currently enrolled in a master's program?,"Where did you do your master's degree? (city, state/region, country).",At what institution did you do your master's degree?,"Master's research topic area (3-5 words e.g. population, methodology and brain region/cognitive function)?",Do you have a doctoral degree or are you currently enrolled in a PhD program?,"Where did you do your doctoral degree? (city, state/region, country).",At what institution did you do your doctoral degree?,"Doctoral research topic area (3-5 key words e.g. population, methodology and brain region/cognitive function)?",Do you have any post-doc / post graduate experience?,"Where did you do your post-doc (city, state/region, country)?",At what institution did you do your post-doc?,"Post-doc research topic area (three key words e.g. population, methodology and brain region/cognitive function)?",Are you a faculty member?,"Where are you a faculty member (city, state/region, country)?",At what institution are you a faculty member?,"Faculty research topic area (three key words e.g. population, methodology and brain region/cognitive function)?",What is your Google Scholar url?
0,2019/08/06 12:34:12 pm GMT-6,Yes,Antonija Kolobaric,"Split, Croatia",1994.0,Female,Experimental Psychology,"Providence, RI, USA",Brown University,"Theory of mind, suicide, development, adolesce...",Yes,"New Haven, CT, USA",Yale University,"early course psychosis, thalamus, mental illne...",No,NaN,NaN,NaN,Yes,"Pittsburgh, PA, USA",University of Pittsburgh,"development, psychosis, fMRI, thalamus",No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,2019/08/06 12:37:29 pm GMT-6,Yes,Giacomo,"Verona, Veneto, Italy",1988.0,Male,Cognitive Psychology,"Rovereto, Trentino, Italy",Unversity of Trento,"healthy, behavioral tracking with IR markers",No,NaN,NaN,NaN,Yes,"Rovereto, Trentino, Italy",University of Trento,"healthy, eyetracking on pointlight displays",Yes,"Rovereto, Trentino, Italy",University of Trento,"healthy, fMRI, movement decoding",Yes,"London, Ontario, Canada",Western University,"healthy, fMRI, skill learning, motor represent...",No,NaN,NaN,NaN,https://scholar.google.com/citations?user=IYbg...
2,2019/08/06 12:38:08 pm GMT-6,Yes,Hannah Coyle,"Northampton, United Kingdom",1991.0,Female,Psychology,"Wollongong, NSW, Australia","University of Wollongong, Australia","cannabis use, EEG, working memory",Yes,"Melbourne, Victoria, Australia",Monash University,"depression, TMS, DLPFC",No,NaN,NaN,NaN,Yes,"Melbourne, Victoria, Australia",Monash University,"mild traumatic brain injury, TMS-EEG, neuropsy...",No,NaN,NaN,NaN,No,NaN,NaN,NaN,https://scholar.google.com/citations?view_op=l...
3,2019/08/06 12:39:18 pm GMT-6,Yes,JKL,Hong Kong,1986.0,Male,Psychology,Hong Kong,University of Hong Kong,"sexual health, clinical, survey",Yes,"Birmingham, UK",University of Birmingham,"stroke, cognitive assessment battery",Yes,"Birmingham, UK",University of Birmingham,"developmental disabilities, eating, cognitive ...",Yes,"Birmingham, UK",University of Birmingham,"temporal, parietal a

In [6]:
# #clean data
# # rename column questions to shorter
df.columns = ['timestamp','consent','name','birthplace','birth_year', 'gender',
              'undergrad_deg','undergrad_loc',  'undergrad_inst','undergrad_research',
              'ra_qual','ra_lm_loc', 'ra_lm_inst', 'ra_lm_research',
              'masters_qual','masters_loc', 'masters_inst', 'masters_research', 
              'phd_qual','phd_loc', 'phd_inst', 'phd_research',
              'post_doc_qual', 'postdoc_loc' ,'postdoc_inst','postdoc_research', 
            'faculty_qual', 'faculty_loc', 'faculty_inst', 'faculty_research',
              'google_scholar']

# change white spaces (blanks) to NaN
#df.replace(r'^\s+$', np.nan, regex=True)

replace_dict_undergrad = {
    "Tijuana, Baja California, MÌ©xico": "Tijuana, Baja California, Mexico",
    'Raleigh & Chapel Hill, North Carolina, USA': 'Chapel Hill, North Carolina, USA'
}
df['undergrad_loc'] = df['undergrad_loc'].replace(replace_dict_undergrad) 

replace_dict_research = {
    'Central Institute of Chemistry and Mechanics': 'Nagatinskaya, Moscow, Russia'
}
df['ra_lm_loc'] = df['ra_lm_loc'].replace(replace_dict_research)

replace_dict_phd = {
    'Rio de Janeiro, Brazil AND Montreal, Canada': 'Rio de Janeiro, Brazil'
}
df['phd_loc'] = df['phd_loc'].replace(replace_dict_phd) 

In [16]:
#Loop through rows to get the longitude and latitude of the hometowns
lat=[]
long=[]
for home_location in df.loc[:,"birthplace"]:
    location_1 = geolocator.geocode(home_location, timeout=10)
    #print((location_1.latitude, location_1.longitude))
    
    lat.append(location_1.latitude)
    long.append(location_1.longitude)

#Saving lat and long in separate columns in the dataframe    
df['HometownLatitude'] = lat
df['HometownLatitude'] = df['HometownLatitude'].astype('float')

df['HometownLongitude'] = long
df['HometownLatitude'] = df['HometownLongitude'].astype('float')
#df.head(20)

In [8]:
#Get the column with hometown location
locs = df.loc[:,"birthplace"]

#Plot hometown locations on the map!
birth_map = folium.Map()

#(location=[location_1.latitude, location_1.longitude])

#Loop through locations and add the markers on the map
for home_location in range(len(locs)): 
    folium.Marker([lat[home_location], long[home_location]], popup=locs[home_location]).add_to(birth_map)

birth_map

In [ ]:
#Try plotting the same but with different markers (circles!)
locs = df.loc[:,"birthplace"]
#Plot hometown locations on the map!
m = folium.Map()
#Loop through locations and add the markers on the map
for home_location in range(len(locs)):
   folium.Circle(
   radius=300,
   location=[lat[home_location], long[home_location]],
   popup=locs[home_location],
   color='crimson',
   fill=False,
).add_to(m)
m

In [11]:
#Loop through rows to get the longitude and latitude of the undergrad_cities (be aware of missing data)
lat_list = []
long_list = []
for undergrad_location in df['undergrad_loc']:
    #print(location_1)
    
    if pd.isnull(undergrad_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(undergrad_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,location))
        
# #Saving lat and long in separate columns in the dataframe    
df['UndergradLatitude' ] = lat_list
df['UndergradLatitude'] = df['UndergradLatitude'].astype('float')

df['UndergradLongitude'] = long_list
df['UndergradLatitude'] = df['UndergradLongitude'].astype('float')
#df.head(40)

In [12]:
# create a map with undergrad locations
df_undergrad = df.loc[:,"UndergradLatitude":"UndergradLongitude"] #create a subset of df to deal with the na problem
df_undergrad = df_undergrad.dropna()
undergrad_locs = df.loc[:,"undergrad_loc"]
#print(df_undergrad)

if None in lat_list: lat_list.remove(None)
if None in long_list: long_list.remove(None)
undergrad_locs = [x for x in undergrad_locs if pd.notna(x)]

In [13]:
# check formats are correct with the NA values removed
print(lat_list)
print(long_list)
print(undergrad_locs)
type(undergrad_locs)

[41.8239891, 45.886548, -34.4243941, 22.2793278, 42.2681569, 38.9719384, -34.6075616, 55.7504461, 18.9387711, 37.4443293, 40.4258686, 32.527002, 25.0375198, 29.8693496, 42.0128695, 37.5666791, 32.5010188, 52.1518157, 36.1556805, 33.5778631, 34.0536909, 33.7872568, -22.9110137, 45.421106, 31.778345, 35.9131542, 55.0282171, 29.9499323, 43.0821793, 35.7006177, 40.7127281, 38.8339578, 43.653963, 31.778345, 43.157285, 34.0966764, 51.4893335]
[-71.4128343, 11.0452369, 150.89385, 114.1628131, -83.7312291, -95.2359496, -58.437076, 37.6174943, 72.8353355, -122.1598465, -86.9080655, -85.4367484053398, 121.5636796, 77.8902124, -73.9081901, 126.9782914, -116.9646629, 4.48110886662043, -95.9929113, -101.8551665, -118.2427666, -117.8503088, -43.2093727, -75.690308, 35.2250786, -79.05578, 82.9234509, -90.0701156, -73.7853915, 51.4013785, -74.0060152, -104.8253485, -79.387207, 35.2250786, -77.615214, -117.7197785, -0.144055084527687]
['Providence, RI, USA', 'Rovereto, Trentino, Italy', 'Wollongong, NS

list

In [14]:
# undergrad locations plotting 
undergrad_map = folium.Map()
#Loop through locations and add the markers on the map
for undergrad_location in range(len(undergrad_locs)): 
    folium.Marker([lat_list[undergrad_location], long_list[undergrad_location]], popup=undergrad_locs[undergrad_location]).add_to(undergrad_map)
    
#display map
undergrad_map

In [ ]:
# create RA locations 
# Loop through rows to get the longitude and latitude of the RA_locations (be aware of missing data)
lat_list = []
long_list = []
for research_location in df['ra_lm_loc']:
    #print(location_1)
    
    if pd.isnull(research_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(research_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,research_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['RA_LM_Latitude' ] = lat_list
df['RA_LM_Latitude'] = df['RA_LM_Latitude'].astype('float')

df['RA_LM_Longitude'] = long_list
df['RA_LM_Latitude'] = df['RA_LM_Longitude'].astype('float')
#df.head(40)


In [ ]:
# create a map with research position locations
df_research = df.loc[:,"RA_LM_Latitude":"RA_LM_Longitude"] #create a subset of df to deal with the na problem
df_research = df_research.dropna()
research_locs = df.loc[:,"ra_lm_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
research_locs = [x for x in research_locs if pd.notna(x)]

In [ ]:
# check formats are correct with the NA values removed
print(lat_list)
print(long_list)
print(research_locs)
type(research_locs)

In [ ]:
# research locations plotting 
research_map = folium.Map()
#Loop through locations and add the markers on the map
for research_location in range(len(research_locs)): 
    folium.Marker([lat_list[research_location], long_list[research_location]], popup=research_locs[research_location]).add_to(research_map)

#display map
research_map

In [ ]:
# create PHD/Doctoral locations 
# Loop through rows to get the longitude and latitude of the RA_locations (be aware of missing data)
lat_list = []
long_list = []
for phd_location in df['phd_loc']:
    #print(location_1)
    
    if pd.isnull(phd_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(phd_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    print((lat,long,phd_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['PHD_Latitude' ] = lat_list
df['PHD_Latitude'] = df['PHD_Latitude'].astype('float')

df['PHD_Longitude'] = long_list
df['PHD_Latitude'] = df['PHD_Longitude'].astype('float')
#df.head(40)

# create a map with research position locations
df_phd = df.loc[:,"PHD_Longitude":"PHD_Latitude"] #create a subset of df to deal with the na problem
df_phd = df_phd.dropna()
phd_locs = df.loc[:,"phd_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
phd_locs = [x for x in phd_locs if pd.notna(x)]




In [ ]:
# research locations plotting 
phd_map = folium.Map()
#Loop through locations and add the markers on the map
for phd_location in range(len(phd_locs)): 
    folium.Marker([lat_list[phd_location], long_list[phd_location]], popup=phd_locs[phd_location]).add_to(phd_map)

#display map    
phd_map

In [43]:
# Plotting several things on one map
f = folium.Figure(width=1000, height=600)
common_map = folium.Map(zoom_start=2, min_zoom=2).add_to(f)

#common_map = folium.Map(zoom_start=1000)

locs = df.loc[:,"birthplace"]

# locs #hometown
# undergrad_locs #undergrad
# phd_locs #phd



#Loop through locations and add the markers on the map
for home_location in range(len(locs)):
   folium.Circle(
   radius=300,
   location=[lat[home_location], long[home_location]],
   popup=locs[home_location],
   color='crimson',
   fill=False,
).add_to(common_map)

for undergrad_location in range(len(undergrad_locs)):
   folium.Circle(
   radius=300,
   location=[lat_list[undergrad_location], long_list[undergrad_location]],
   popup=undergrad_locs[undergrad_location],
   color='blue',
   fill=False,
).add_to(common_map)

common_map
# range(len(locs))
# type(lat)

f.save('map.html')

f

In [ ]:
#Plot 2 maps separately

In [ ]:
#Playing with ipyleaflet
# from ipyleaflet import (Map)

In [ ]:
# #Gif map!
# import matplotlib.pyplot as plt
# import geopandas as gpd

# # save all the maps in the charts folder
# output_path = 'charts/maps'

# # counter for the for loop
# i = 0

# # list of years (which are the column names at the moment)
# career_stages = ['Born','Undergrad','RA','PhD','Postdoc','Faculty']

In [ ]:
#Trying to add feature clusters!!!
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium.element import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps

def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
crime_map = add_point_clusters(common_map, assaults, ['Hometown','Undergrad'])

folium.LayerControl().add_to(common_map) #Add layer control to toggle on/off
common_map.save('hometown-undergrad.html') #save HTML
common_map #display map